# Anveshak Rathore
## 181060012
## Final Year B.Tech Electronics

In [1]:
from IPython.display import clear_output
from time import sleep
import gym
import numpy as np
import random
from tqdm import tqdm

In [2]:
parking = gym.make("Taxi-v3").env
parking.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## The <span style="color:blue">blue letter</span> denotes the location of the **passenger**.
## The <span style="color:purple">purple letter</span> denotes the location of the **destination**.
## A <span style="color:yellow">yellow box</span> denotes the **passengerless** taxicab.
## A <span style="color:green">green box</span> denotes the taxicab **with a passenger**.
## A **|** represents a wall that the taxi cannot cross.

In [3]:
print("Action space: ",parking.action_space)
print("State space: ",parking.observation_space)

Action space:  Discrete(6)
State space:  Discrete(500)


In [4]:
print("Format of reward table:")
print("action: probability, next state, reward, done")
parking.P[parking.s]

Format of reward table:
action: probability, next state, reward, done


{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

In [5]:
def animator(frames,delay):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(delay)
    return

# **Attempting via Brute Force method**

In [7]:
epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done:
    action = parking.action_space.sample()
    state, reward, done, info = parking.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': parking.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 9024
Penalties incurred: 2905


# **Visualization of attempt**

In [10]:
animator(frames,0.05)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| |_: | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 532
State: 336
Action: 3
Reward: -1


KeyboardInterrupt: 

# **Attempting via Q-Reinforcement method**

## Training the agent

In [8]:
q_table = np.zeros([parking.observation_space.n, parking.action_space.n])

alpha = 0.5     # LEARNING RATE
gamma = 0.5     # DISCOUNT FACTOR
epsilon = 0.5   # EXPLORATION PARAMETER

all_epochs = []
all_penalties = []

for _ in tqdm(range(1, 100001), desc="Episode: "):
    state = parking.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = parking.action_space.sample()
        else:
            action = np.argmax(q_table[state]) 

        next_state, reward, done, info = parking.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

print("Training finished.\n")

Episode: 100%|██████████| 100000/100000 [00:57<00:00, 1743.69it/s]

Training finished.



## Evaluating the agent

In [9]:
total_epochs, total_penalties = 0, 0
episodes = 10
total_frames = {}
temp = []

for i in range(episodes):
    state = parking.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = parking.step(action)

        if reward == -10:
            penalties += 1

        temp.append({
                'frame': parking.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                })

        epochs += 1
    total_frames[i] = temp
    #animator(total_frames,0.5)

    #sleep(3)

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 10 episodes:
Average timesteps per episode: 12.6
Average penalties per episode: 0.0


## **Visualization of attempt**

In [11]:
for i in total_frames:
    animator(total_frames[i],0.5)
    sleep(3)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 108
State: 202
Action: 1
Reward: -1


KeyboardInterrupt: 